# **The fourth in-class-exercise (40 points in total, 11/03/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
# Write your code here

# Importing modules
import pandas as pd

papers = pd.read_csv('papers.csv')
papers.head()


,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [ ]:
# Remove the columns
papers = papers.drop(columns=['id', 'event_type', 'pdf_name'], axis=1).sample(100)

papers.head()

,year,title,abstract,paper_text
4507,2013,Statistical analysis of coupled time series wi...,Many applications require the analysis of comp...,Statistical analysis of coupled time series wi...
3549,2011,ShareBoost: Efficient multiclass learning with...,Multiclass prediction is the problem of classi...,ShareBoost: Efficient Multiclass Learning with...
5993,2016,Bayesian Optimization for Probabilistic Programs,We present the first general purpose framework...,Bayesian Optimization for Probabilistic Progra...
3629,2011,Inference in continuous-time change-point models,We consider the problem of Bayesian inference ...,Inference in continuous-time change-point mode...
5088,2014,Learning Shuffle Ideals Under Restricted Distr...,The class of shuffle ideals is a fundamental s...,Learning Shuffle Ideals\nUnder Restricted Dist...


In [ ]:
# Load the regular expression library
import re
# Remove punctuation
papers['paper_text_processed'] = \
papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['paper_text_processed'] = \
papers['paper_text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['paper_text_processed'].head()

4507    statistical analysis of coupled time series wi...
3549    shareboost: efficient multiclass learning with...
5993    bayesian optimization for probabilistic progra...
3629    inference in continuous-time change-point mode...
5088    learning shuffle ideals\nunder restricted dist...
Name: paper_text_processed, dtype: object

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maganti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['statistical', 'analysis', 'coupled', 'time', 'series', 'kernel', 'cross', 'spectral', 'density', 'operators', 'michel', 'besserve', 'mpi', 'intelligent', 'systems', 'mpi', 'biological', 'cybernetics', 'tubingen', 'germany', 'michelbesserve', 'tuebingenmpgde', 'nikos', 'logothetis', 'mpi', 'biological', 'cybernetics', 'tubingen', 'nikoslogothetis', 'tuebingenmpgde']


In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 4), (6, 1), (7, 2), (8, 3), (9, 1), (10, 1), (11, 2), (12, 3), (13, 5), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 7), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 3), (27, 2), (28, 3), (29, 1)]


In [ ]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.007*"data" + 0.006*"learning" + 0.005*"model" + 0.005*"algorithm" + '
  '0.004*"using" + 0.004*"time" + 0.004*"set" + 0.003*"two" + 0.003*"neural" + '
  '0.003*"function"'),
 (1,
  '0.009*"learning" + 0.005*"function" + 0.005*"one" + 0.004*"data" + '
  '0.004*"algorithm" + 0.004*"problem" + 0.004*"model" + 0.004*"training" + '
  '0.004*"using" + 0.003*"set"'),
 (2,
  '0.007*"learning" + 0.006*"model" + 0.005*"using" + 0.004*"information" + '
  '0.004*"set" + 0.004*"data" + 0.003*"algorithm" + 0.003*"function" + '
  '0.003*"time" + 0.003*"one"'),
 (3,
  '0.007*"learning" + 0.006*"algorithm" + 0.006*"data" + 0.005*"function" + '
  '0.004*"set" + 0.004*"using" + 0.004*"model" + 0.004*"one" + '
  '0.004*"distribution" + 0.004*"figure"'),
 (4,
  '0.007*"learning" + 0.006*"model" + 0.005*"data" + 0.004*"one" + '
  '0.004*"using" + 0.003*"function" + 0.003*"problem" + 0.003*"two" + '
  '0.003*"set" + 0.003*"based"'),
 (5,
  '0.006*"learning" + 0.005*"function" + 0.004*"model" + 0.00

In [ ]:
#import pyLDAvis.gensim

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()


LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

pyLDAvis.display(LDAvis_prepared)

C:\Users\maganti\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
# Write your code here
import pandas as pd

# load data
df = pd.read_csv('Musical_instruments_reviews.csv', usecols=['reviewerID', 'reviewText'])
df.head()

,reviewerID,reviewText
0,A2IBPI20UZIR0U,"Not much to write about here, but it does exac..."
1,A14VAT5EAX3D9S,The product does exactly as it should and is q...
2,A195EZSQDW3E21,The primary job of this device is to block the...
3,A2C00NNG1ZQQG2,Nice windscreen protects my MXL mic and preven...
4,A94QU4C90B1AX,This pop filter is great. It looks and perform...


In [ ]:
print(df.shape)
df = df.dropna()
print(df.shape)

(10261, 2)
(10254, 2)


In [ ]:
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation \
                                        , preprocess_string, strip_short, stem_text

# preprocess given text
def preprocess(text):
    
    # clean text based on given filters
    CUSTOM_FILTERS = [lambda x: x.lower(), 
                                remove_stopwords, 
                                strip_punctuation, 
                                strip_short, 
                                stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)
    
    return text

# apply function to all reviews 
df['Text (Clean)'] = df['reviewText'].apply(lambda x: preprocess(x))

In [ ]:
from gensim import corpora

# create a dictionary with the corpus
corpus = df['Text (Clean)']
dictionary = corpora.Dictionary(corpus)

# convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in corpus]

In [ ]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=df['Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 2 clusters: 0.45969944165248133
Coherence score with 3 clusters: 0.43605470300473614
Coherence score with 4 clusters: 0.3860662019606791
Coherence score with 5 clusters: 0.43244266091048145
Coherence score with 6 clusters: 0.37003148709574935
Coherence score with 7 clusters: 0.3975086651330142
Coherence score with 8 clusters: 0.3746734174298496
Coherence score with 9 clusters: 0.397930908204572
Coherence score with 10 clusters: 0.3953288301432295


In [ ]:
lsi = LsiModel(bow, num_topics=10, id2word=dictionary)

for topic_num, words in lsi.print_topics(num_words=5):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.329*"sound" + 0.314*"guitar" + 0.242*"string" + 0.232*"pedal" + 0.217*"amp".
Words in 1: 0.584*"string" + -0.428*"pedal" + 0.380*"guitar" + -0.312*"amp" + 0.161*"tune".
Words in 2: -0.529*"string" + -0.367*"pedal" + 0.261*"mic" + 0.190*"record" + -0.187*"tone".
Words in 3: -0.641*"pedal" + 0.520*"amp" + 0.306*"sound" + -0.178*"guitar" + 0.119*"mic".
Words in 4: 0.641*"guitar" + 0.351*"amp" + -0.342*"string" + -0.269*"mic" + -0.192*"record".
Words in 5: -0.460*"tuner" + 0.382*"sound" + -0.372*"tune" + -0.309*"amp" + 0.246*"guitar".
Words in 6: 0.848*"pick" + -0.214*"guitar" + 0.172*"plai" + 0.155*"like" + -0.119*"sound".
Words in 7: 0.426*"sound" + -0.384*"strap" + 0.359*"tuner" + -0.350*"cabl" + 0.307*"tune".
Words in 8: 0.450*"cabl" + -0.436*"stand" + -0.399*"mic" + 0.324*"sound" + 0.231*"good".
Words in 9: -0.670*"strap" + 0.355*"cabl" + 0.273*"guitar" + -0.181*"like" + -0.181*"sound".


## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
# Write your code here
from lda2vec import preprocess, Corpus
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

try:
    import seaborn
except:
    pass



In [ ]:
import pyLDAvis
pyLDAvis.enable_notebook()

In [ ]:
npz = np.load(open('topics.pyldavis.npz', 'r'))
dat = {k: v for (k, v) in npz.iteritems()}
dat['vocab'] = dat['vocab'].tolist()


top_n = 10
topic_to_topwords = {}
for j, topic_to_word in enumerate(dat['topic_term_dists']):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    top_words = [dat['vocab'][i].strip()[:35] for i in top]
    msg += ' '.join(top_words)
    print msg
    topic_to_topwords[j] = top_words

In [ ]:
import warnings
warnings.filterwarnings('ignore')
prepared_data = pyLDAvis.prepare(dat['topic_term_dists'], dat['doc_topic_dists'], 
                                 dat['doc_lengths'] * 1.0, dat['vocab'], dat['term_frequency'] * 1.0, mds='tsne')

In [ ]:
from sklearn.datasets import fetch_20newsgroups
remove=('headers', 'footers', 'quotes')
texts = fetch_20newsgroups(subset='train', remove=remove).data

In [ ]:
msg = "{weight:02d}% in topic {topic_id:02d} which has top words {text:s}"
for topic_id, weight in enumerate(dat['doc_topic_dists'][1]):
    if weight > 0.01:
        text = ', '.join(topic_to_topwords[topic_id])
        print msg.format(topic_id=topic_id, weight=int(weight * 100.0), text=text)

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# Write your code here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('/content/gdrive/My Drive/Assignment/tokyo_2020_tweets.csv')
df = df.dropna()
df = df.sample(20000, random_state=42)
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
295003,1419957058938486804,Australia in Belgium 🇦🇺🇪🇺🇧🇪🇱🇺,"Brussels, Belgium",Australian Mission to the European Union and N...,2014-11-26 15:52:22,3766,3226,4622,True,2021-07-27 09:45:19,"Congrats to 🇦🇺, 🇧🇪, and 🇱🇺 for our achievement...",['Tokyo2020'],Twitter Web App,0.0,0.0,False
63678,1419307203933229065,念真 Nian Zhen,"Enshi, Hubei, China.",一个中国\n\nChina institute of contemporary intern...,2017-06-30 20:07:29,7788,87.0,38102.0,False,2021-07-25 14:43:01,Reality Vs New York Times\n#To...,['Tokyo2020'],Twitter for Android,12.0,57.0,False
101824,1419592988833947650,Indonesia Badminton,Indonesia,Indonesia Badminton Supporter • Click to follo...,2009-11-07 23:26:53,55711,451,2081,False,2021-07-26 09:38:37,[Live Score] #Tokyo2020 \n\nWD Group A : \n\...,['Tokyo2020'],Twitter for iPhone,0.0,4.0,False
60985,1419311331782447108,FirstSportz,New Delhi,Official Twitter handle of Firstsportz. \nFoll...,2019-11-26 14:14:55,478,258.0,4053.0,False,2021-07-25 14:59:25,Tennis at Tokyo Olympics: Stefanos #Tsitsipas ...,['Tsitsipas'],Twitter for Android,0.0,0.0,False
47225,1419235863100370946,Kurdistan 24 English,"Erbil, Kurdistan",The English service of Kurdistan's leading new...,2015-07-06 08:32:11,139803,5.0,7.0,True,2021-07-25 09:59:32,The 41-year-old gold medalist is an ethnic Kur...,"['Tokyo2020', 'Olympics']",Twitter Web App,3.0,9.0,False


In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

def clean_text(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
  x = re.sub(r'https*://.*', ' ', x)
  x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
  x = re.sub(r'\d+', ' ', x)
  x = re.sub(r'\n+', ' ', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x


df['clean_text'] = df.text.apply(clean_text)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,clean_text
295003,1419957058938486804,Australia in Belgium 🇦🇺🇪🇺🇧🇪🇱🇺,"Brussels, Belgium",Australian Mission to the European Union and N...,2014-11-26 15:52:22,3766,3226,4622,True,2021-07-27 09:45:19,"Congrats to 🇦🇺, 🇧🇪, and 🇱🇺 for our achievement...",['Tokyo2020'],Twitter Web App,0.0,0.0,False,congrats 🇦🇺 🇧🇪 🇱🇺 achievements far olympics 👏👏👏
63678,1419307203933229065,念真 Nian Zhen,"Enshi, Hubei, China.",一个中国\n\nChina institute of contemporary intern...,2017-06-30 20:07:29,7788,87.0,38102.0,False,2021-07-25 14:43:01,Reality Vs New York Times\n#To...,['Tokyo2020'],Twitter for Android,12.0,57.0,False,reality vs new york times
101824,1419592988833947650,Indonesia Badminton,Indonesia,Indonesia Badminton Supporter • Click to follo...,2009-11-07 23:26:53,55711,451,2081,False,2021-07-26 09:38:37,[Live Score] #Tokyo2020 \n\nWD Group A : \n\...,['Tokyo2020'],Twitter for iPhone,0.0,4.0,False,live score wd group greysia polii apriyani ra...
60985,1419311331782447108,FirstSportz,New Delhi,Official Twitter handle of Firstsportz. \nFoll...,2019-11-26 14:14:55,478,258.0,4053.0,False,2021-07-25 14:59:25,Tennis at Tokyo Olympics: Stefanos #Tsitsipas ...,['Tsitsipas'],Twitter for Android,0.0,0.0,False,tennis tokyo olympics stefanos becomes first g...
47225,1419235863100370946,Kurdistan 24 English,"Erbil, Kurdistan",The English service of Kurdistan's leading new...,2015-07-06 08:32:11,139803,5.0,7.0,True,2021-07-25 09:59:32,The 41-year-old gold medalist is an ethnic Kur...,"['Tokyo2020', 'Olympics']",Twitter Web App,3.0,9.0,False,year old gold medalist ethnic kurd western ir...


In [7]:
! pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 6.9 MB/s 
     |████████████████████████████████| 85 kB 4.7 MB/s 
     |████████████████████████████████| 5.2 MB 40.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 636 kB 44.4 MB/s 
     |████████████████████████████████| 88 kB 7.0 MB/s 
     |████████████████████████████████| 5.5 MB 28.8 MB/s 
     |████████████████████████████████| 1.3 MB 33.3 MB/s 
     |████████████████████████████████| 163 kB 49.6 MB/s 
     |████████████████████████████████| 7.6 MB 60.0 MB/s 
     |████████████████████████████████| 1.1 MB 45.4 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp37-cp37m-linux_x86_64.whl size=2340736 sha256=c668db4a9326a0e261d150be0504fc7f7b541c3f4f50414bfa39e0059e496089
  Stored in directory: /root/.cache

In [8]:
from bertopic import BERTopic
tweets = df.clean_text.to_list()
timestamp = df.date.to_list()

topic_model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(tweets)

topic_model.get_topic_info()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

,Topic,Count,Name
0,-1,6569,-1_day_athletes_today_game
1,0,315,0_olympics_olympic_games_athletes
2,1,288,1_medals_medal_silver_gold
3,2,257,2_garay_femenino_tandara_de
4,3,203,3_air_pistol_saurabh_chaudhary
...,...,...,...
357,356,10,356_half_fifty_dozen_split
358,357,10,357_news_headline_predicting_glimpse
359,358,10,358_pen_ta_owed_tattoo
360,359,10,359_yellow_card_picks_suspension


In [11]:
topic_model.get_topic_info(0)

,Topic,Count,Name
0,0,315,0_olympics_olympic_games_athletes


In [9]:
topic_model.visualize_topics()

In [10]:
topic_model.visualize_barchart(top_n_topics=12, n_words = 10, width = 350, height = 350)

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)



The results of LDA are as follows:
'0.007*"data" + 0.006*"learning" + 0.005*"model" + 0.005*"algorithm" + '
  '0.004*"using" + 0.004*"time" + 0.004*"set" + 0.003*"two" + 0.003*"neural" + '
  '0.003*"function"

The results of LSA are as follows:
0.329*"sound" + 0.314*"guitar" + 0.242*"string" + 0.232*"pedal" + 0.217*"amp".

The results of lda2vec are as follows:
topic 00 which has top words x11r5, xv, window, xterm, server, motif, font, xlib, //, sunos

The results of Bertopic are as follows:
olympics_olympic_games_athletes_olympians_watching sports_summer


Bertopic results are more easily interpretable.